# Imports

In [ ]:
import os
import numpy as np
import subprocess
import pandas as pd
import requests
from urllib.parse import urljoin
import sqlite3
import json
import time
from datetime import datetime, timedelta
import re
from google.colab import drive
import random
from bs4 import BeautifulSoup
import csv

drive.mount('/content/gdrive/', force_remount=True)
os.chdir("/content/gdrive/Shareddrives/ECS 260/final")

Mounted at /content/gdrive/


# Exploring counts and values of some metrics

In [ ]:
def get_df(query, conn):
  return pd.read_sql_query(query, conn)

def get_distinct_values_from_column(conn, column_name, table = 'package_data'):
  sql_query = f"SELECT DISTINCT {column_name} FROM {table}"
  cursor.execute(sql_query)
  rows = cursor.fetchall()
  return [row[0] for row in rows]

def get_valid_values_of_column(conn, column_name, table = 'package_data'):
  sql_query = f"SELECT * FROM {table} WHERE {column_name} IS NOT NULL"
  cursor.execute(sql_query)
  rows = cursor.fetchall()
  return [row[0] for row in rows]

def get_column_names(conn, table = 'package_data'):
  cursor.execute(f"PRAGMA table_info({table})")
  return [column[1] for column in cursor.fetchall()]

In [ ]:
conn = sqlite3.connect("final_database.db")
cursor = conn.cursor()

In [ ]:
# sql_query = """
#     SELECT * FROM package_data
#     WHERE contributors = '5000+';
# """
# # cursor.execute("UPDATE package_data SET contributors=16979 WHERE git_repository_url_final='https://github.com/DefinitelyTyped/DefinitelyTyped'")
# # conn.commit()
# df = get_df(sql_query, conn)
# set(df['git_repository_url_final'])

In [ ]:
# get table names in the database
get_df("SELECT name FROM sqlite_master WHERE type='table'", conn)

,name
0,package_data
1,download_data
2,package_metrics
3,temp_table
4,metric_analysis


In [ ]:
df = get_df("SELECT * FROM metric_analysis", conn)
df

,package,is_deprecated,dependants_count,forks,issues,pr,contributors,unpack_size,total_lines_of_code,sloc,days_since_last_modification,avg_commit_freq,threat_score,file_count,state
0,@gerrico/react-components,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,express-simple-app-generator,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,generator-giraffe,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,outdated-client,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,@semi-bot/semi-theme-shopify,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,haribotify,None,None,None,None,None,None,None,None,None,None,None,None,None,None
29996,eslint-config-sharecar,None,None,None,None,None,None,None,None,None,None,None,None,None,None
29997,webpack-to-ardoq,None,None,None,None,None,None,None,None,None,None,None,None,None,None
29998,zywave-content-search,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
df = get_df("SELECT DISTINCT package FROM package_data", conn)
print('Total unique rows present:', len(df))

Total unique rows present: 30000


In [ ]:
sql_query = """
    SELECT * FROM package_data
    WHERE npm_api_status = '200';
"""
df = get_df(sql_query, conn)
print('Total valid packages mined:', len(df))
# df

Total valid packages mined: 29792


In [ ]:
sql_query = """
    SELECT * FROM package_data
    WHERE npm_api_status = '404';
"""
df = get_df(sql_query, conn)
print('Total packages not found:', len(df))
# df

Total packages not found: 208


In [ ]:
df = get_valid_values_of_column(conn, 'unpublished_timestamp')
print('Total packages that have been unpublished:', len(df))
# df

Total packages that have been unpublished: 342


In [ ]:
sql_query = """
    SELECT * FROM package_data
    WHERE is_deprecated = '1';
"""
df = get_df(sql_query, conn)
print('Total packages that have been deprecated:', len(df))
# df

Total packages that have been deprecated: 1025


In [ ]:
df = get_valid_values_of_column(conn, 'lagging_dependencies_count')
print('Total packages that have some value in lagging_dependencies_count:', len(df))
# df

Total packages that have some value in lagging_dependencies_count: 29450


In [ ]:
df = get_valid_values_of_column(conn, 'git_repository_url')
print('Total packages that were able to give a github repo url:', len(df))
# df

Total packages that were able to give a github repo url: 16859


In [ ]:
print('Distinct values of the column git_repository_status')
get_distinct_values_from_column(conn, 'git_repository_status')

Distinct values of the column git_repository_status


['200', None, '404']

In [ ]:
sql_query = """
    SELECT * FROM package_data
    WHERE git_repository_status = '404';
"""
df = get_df(sql_query, conn)
print('Total packages that have invalid github url(repository not found):', len(df))
# df

Total packages that have invalid github url(repository not found): 3433


In [ ]:
sql_query = """
    SELECT * FROM package_data
    WHERE git_repository_status = '200';
"""
df = get_df(sql_query, conn)
print('Total packages that have valid github url:', len(df))
# df

Total packages that have valid github url: 13450


In [ ]:
sql_query = """
    SELECT * FROM package_data
    WHERE file_count IS NOT NULL;
"""
df = get_df(sql_query, conn)
print('Total packages that have valid file_count(from npm API):', len(df))
# df
sql_query = """
    SELECT * FROM package_data
    WHERE no_of_files IS NOT NULL;
"""
df = get_df(sql_query, conn)
print('Total packages that have valid file_count(from pydrilling github repo):', len(df))
# df

Total packages that have valid file_count(from npm API): 23926
Total packages that have valid file_count(from pydrilling github repo): 13450


In [ ]:
df1 = get_valid_values_of_column(conn, 'stars')
print('Total packages that have valid stars:', len(df1))
df = get_valid_values_of_column(conn, 'forks')
print('Total packages that have valid forks:', len(df))
df = get_valid_values_of_column(conn, 'watchers')
print('Total packages that have valid watchers:', len(df))
df = get_valid_values_of_column(conn, 'issues')
print('Total packages that have valid issues:', len(df))
df = get_valid_values_of_column(conn, 'pr')
print('Total packages that have valid pr:', len(df))
df = get_valid_values_of_column(conn, 'contributors')
print('Total packages that have valid contributors:', len(df))
df = get_valid_values_of_column(conn, 'no_of_commits')
print('Total packages that have valid no_of_commits:', len(df))
df2 = get_valid_values_of_column(conn, 'avg_commit_freq')
print('Total packages that have valid avg_commit_freq:', len(df2))
df2 = get_valid_values_of_column(conn, 'avg_commit_freq')
print('Total packages with empty repository:', len(df1) - len(df2))
print('Distinct values of the column repository_type: ', get_distinct_values_from_column(conn, 'repository_type'))

Total packages that have valid stars: 13450
Total packages that have valid forks: 13450
Total packages that have valid watchers: 13450
Total packages that have valid issues: 13450
Total packages that have valid pr: 13450
Total packages that have valid contributors: 13405
Total packages that have valid no_of_commits: 13406
Total packages that have valid avg_commit_freq: 13406
Total packages with empty repository: 44
Distinct values of the column repository_type:  ['git', None, 'any', 'http', 'github', 'hg', '', 'git+https', 'vsts', 'private', 'Git', 'https', 'GIT', 'GitHub', 'gitlab', '???', 'git+ssh', 'gitee', 'bae', 'url', 'mercurial', 'darcs']


In [ ]:
print('Distinct values of the column gh_scrapping_status')
print(get_distinct_values_from_column(conn, 'gh_scrapping_status'))
sql_query = """
    SELECT * FROM package_data
    WHERE gh_scrapping_status = '200';
"""
df = get_df(sql_query, conn)
print('Total packages that have valid gh_scrapping_status:', len(df))

Distinct values of the column gh_scrapping_status
['200', None]
Total packages that have valid gh_scrapping_status: 13452


In [ ]:
print('Distinct values of the column contributors', get_distinct_values_from_column(conn, 'contributors'))

Distinct values of the column contributors [1, None, 2, 0, 13, 514, 3, 8, 12, 5, 6, 9, 15, 768, 7, 62, 10, 16979, 4, 51, 16, 721, 91, 65, 24, 11, 89, 14, 217, 28, 56, 30, 39, 29, 92, 19, 165, 98, 85, 22, 187, 190, 23, 21, 20, 26, 40, 47, 18, 27, 213, 897, 60, 31, 134, 25, 113, 34, 52, 105, 37, 132, 112, 102, 42, 94, 58, 135, 203, 64, 35, 228, 373, 59, 109, 182, 394, 17, 43, 568, 2919, 86, 476, 137, 151, 168, 607, 359, 219, 138, 153, 82, 361, 162, 80, 532, 48, 136, 53, 50, 33, 81, 150, 46, 119, 814, 291, 54, 223, 174, 77, 71, 427, 170, 100, 44, 74, 140, 325, 422, 455, 164, 171, 103, 108, 1071, 453, 63, 183, 36, 66, 465, 201, 32, 131, 288, 67, 68, 248, 55, 41, 78, 244, 49, 148, 378, 1380, 214, 157, 298, 38, 668, 250, 72, 87, 406, 61, 622, 1105, 374, 209, 332, 663, 524, 351, 154, 255, 70, 220, 139, 116, 377, 324, 979, 278, 79, 45, 2614, 104, 95, 147, 272, 111, 142, 562, 612, 405, 97, 196, 258, 918, 115, 915, 387, 615, 1228, 126, 121, 96, 603, 125, 580, 795, 101, 454, 287, 3990, 83, 283, 1

In [ ]:
print('Distinct values of the column health_score')
print(get_distinct_values_from_column(conn, 'health_score'))
print('Distinct values of the column security')
print(get_distinct_values_from_column(conn, 'security'))
print('Distinct values of the column popularity')
print(get_distinct_values_from_column(conn, 'popularity'))
print('Distinct values of the column maintenance')
print(get_distinct_values_from_column(conn, 'maintenance'))
print('Distinct values of the column community')
print(get_distinct_values_from_column(conn, 'community'))
df = get_df(sql_query, conn)
print('Distinct values of the column snyk_scrapping_status:\n', get_distinct_values_from_column(conn, 'snyk_scrapping_status'))

Distinct values of the column health_score
[30, 54, 36, 28, 40, 46, 57, 42, 50, 34, 43, None, 61, 89, 0, 48, 49, 55, 58, 37, 52, 53, 72, 45, 39, 44, 69, 56, 63, 67, 22, 41, 47, 59, 51, 74, 70, 31, 60, 73, 64, 79, 95, 65, 71, 84, 86, 80, 68, 38, 66, 83, 62, 82, 76, 77, 75, 78, 90, 87, 92, 88, 81, 85, 91, 20, 35, 24, 93, 33, 97, 98, 18, 100]
Distinct values of the column security
['SECURITY REVIEW NEEDED', 'NO KNOWN SECURITY ISSUES', None, 'SECURITY ISSUES FOUND', '?']
Distinct values of the column popularity
['LIMITED', None, 'INFLUENTIAL PROJECT', 'SMALL', 'RECOGNIZED', 'POPULAR', 'KEY ECOSYSTEM PROJECT']
Distinct values of the column maintenance
['INACTIVE', 'SUSTAINABLE', None, 'HEALTHY']
Distinct values of the column community
['LIMITED', None, 'SUSTAINABLE', 'ACTIVE']
Distinct values of the column snyk_scrapping_status:
 ['200', '404']


In [ ]:
df = get_df("SELECT * from package_data WHERE security = '?'", conn)
df

,package,npm_api_status,latest_version,no_of_versions,keywords,no_of_users,has_readme,has_homepage,last_modified_timestamp,created_timestamp,...,security,popularity,maintenance,community,avg_monthly_download,days_since_last_modification,security_numeric,popularity_numeric,maintenance_numeric,community_numeric
0,kuda-delsia-p2p-binance-axxz5zf0v,200,None,1,None,None,None,None,2023-10-31T15:25:13.847Z,2023-04-21T22:40:05.486Z,...,?,LIMITED,INACTIVE,LIMITED,None,114,None,20,5,20


In [ ]:
df = get_df("SELECT * from package_data WHERE snyk_scrapping_status ='200'", conn)
print('Total packages that were scrapped successfully:', len(df))
df = get_df("SELECT * from package_data WHERE health_score is not null and not health_score = '?'", conn)
print('Total packages that have valid health_score:', len(df))
df1 = get_df("SELECT * from package_data WHERE security is not null and not security = '?' and not security = 'PENDING…'", conn)
print('Total packages that have valid security:', len(df1))
df = get_df("SELECT * from package_data WHERE popularity is not null and not popularity = '?' and not popularity = 'PENDING…'", conn)
print('Total packages that have valid popularity:', len(df))
df = get_df("SELECT * from package_data WHERE maintenance is not null and not maintenance = '?' and not maintenance = 'PENDING…'", conn)
print('Total packages that have valid maintenance:', len(df))
df = get_df("SELECT * from package_data WHERE community is not null and not community = '?' and not maintenance = 'PENDING…'", conn)
print('Total packages that have valid community:', len(df))

Total packages that were scrapped successfully: 29284
Total packages that have valid health_score: 29284
Total packages that have valid security: 28960
Total packages that have valid popularity: 28961
Total packages that have valid maintenance: 28961
Total packages that have valid community: 28961


In [ ]:
# getting columns to analyze
with open('columns_to_analyze.txt', "r") as file:
  columns_to_analyze = file.read()
columns_to_analyze = json.loads(columns_to_analyze)

conditions = [f"{column} IS NOT NULL" for column in columns_to_analyze]
sql_query = f"SELECT * FROM package_data\nWHERE "
sql_query += " AND ".join(conditions) + ";"
df = get_df(sql_query, conn)
print(f"Total {len(df)} rows present with all non-null values")
# df

Total 244 rows present with all non-null values


In [ ]:
print('Distinct values of the column rank')
print(get_distinct_values_from_column(conn, 'rank'))

Distinct values of the column rank
[None, '6', '10', '3', '5', '2', '4', '8', '23', '9', '11', '7', '14', '12', '1', '13', '0', '25', '22', '16', '15', '20', '17', '21', '19', '18', '27', '24', '26', '28', '29']


In [ ]:
print('Distinct values of the dependants_count')
print(get_distinct_values_from_column(conn, 'dependants_count'))

Distinct values of the dependants_count
['3', '0', '1', '2', None, '15', '9', '129561', '1291', '4', '212', '5', '15852850', '77', '24', '518', '25', '24780620', '114', '10', '6', '1088', '70', '14738430', '13', '810', '1633', '24797578', '21', '27', '26', '50', '23', '186', '35', '136', '20537', '40', '14', '22', '7', '52', '24798545', '78', '74', '13517310', '3951', '91817', '346', '626', '16', '32', '2939', '2370', '410809', '409', '28', '25248', '19', '460', '515', '8', '744', '4700', '461400', '87', '227', '31', '7504', '1837', '19321', '39374', '1335', '479', '106', '11', '500', '2577', '11159', '1449543', '82', '3977', '133', '18294548', '41', '20', '18', '1601', '110', '11928462', '15733', '38', '646', '9971542', '7852', '2509', '263', '2805', '262439', '532', '168', '12', '1511', '149', '440', '981', '59', '60', '2115', '80', '16977', '284', '45', '424', '247', '1212', '27764543', '104', '33', '67', '4110', '193', '296', '27311', '743', '264', '43', '290', '1057', '1413', '245